<a href="https://colab.research.google.com/github/CasCard/RIG-Works/blob/main/img_proccessing_hsr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/gdrive', force_remount=True)
from numba import jit

Mounted at /content/gdrive


In [ ]:
import cv2
from skimage.color import rgb2hsv,hsv2rgb
import numpy as np


def  HSVStretching(sceneRadiance):
    sceneRadiance = np.uint8(sceneRadiance)
    height = len(sceneRadiance)
    width = len(sceneRadiance[0])
    img_hsv = rgb2hsv(sceneRadiance)
    h, s, v = cv2.split(img_hsv)
    img_s_stretching = global_stretching(s, height, width)
    img_v_stretching = global_stretching(v, height, width)

    labArray = np.zeros((height, width, 3), 'float64')
    labArray[:, :, 0] = h
    labArray[:, :, 1] = img_s_stretching
    labArray[:, :, 2] = img_v_stretching
    img_rgb = hsv2rgb(labArray) * 255

    # img_rgb = np.clip(img_rgb, 0, 255)

    return img_rgb

In [ ]:
pip install numba

In [ ]:
import numpy as np


def sceneRadianceRGB(sceneRadiance):

    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    sceneRadiance = np.uint8(sceneRadiance)

    return sceneRadiance



In [ ]:
import numpy as np


def global_stretching(img_L,height, width):
    I_min = np.min(img_L)
    I_max = np.max(img_L)
    I_mean = np.mean(img_L)

    array_Global_histogram_stretching_L = np.zeros((height, width))
    for i in range(0, height):
        for j in range(0, width):
            p_out = (img_L[i][j] - I_min) * ((1) / (I_max - I_min))
            array_Global_histogram_stretching_L[i][j] = p_out

    return array_Global_histogram_stretching_L



In [ ]:
import numpy as np
import concurrent.futures
import copy



def histogram_general(r_array,height, width,input):
    length = height * width
    R_rray = []
    for i in range(height):
        for j in range(width):
            R_rray.append(r_array[i][j])
    R_rray.sort()
    I_min = int(R_rray[int(length / 500)])
    I_max = int(R_rray[-int(length / 500)])
    array_Global_histogram_stretching = np.zeros((height, width))
    if input == 0:
      for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = I_min
            elif (r_array[i][j] > I_max):
                p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = 255
            else:
                p_out = int((r_array[i][j] - I_min) * ((255 - I_min) / (I_max - I_min)))+ I_min
                array_Global_histogram_stretching[i][j] = p_out

    elif input == 1:
      for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                  p_out = r_array[i][j]
                  array_Global_histogram_stretching[i][j] = 0
            elif (r_array[i][j] > I_max):
                  p_out = r_array[i][j]
                  array_Global_histogram_stretching[i][j] = 255
            else:
                  p_out = int((r_array[i][j] - I_min) * ((255) / (I_max - I_min)))
                  array_Global_histogram_stretching[i][j] = p_out
    elif input == 2 :
      for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = 0
            elif (r_array[i][j] > I_max):
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = I_max
            else:
                p_out = int((r_array[i][j] - I_min) * ((I_max) / (I_max - I_min)))
                array_Global_histogram_stretching[i][j] = p_out

    return (array_Global_histogram_stretching)

In [ ]:
def stretching(img):
    height = len(img)
    width = len(img[0])
    with concurrent.futures.ThreadPoolExecutor() as executor:
      input=[img[:, :, 2],img[:, :, 1],img[:, :, 0]]
      inp=[0,1,2]
      h=[height,height,height]
      w=[width,width,width]
      results=executor.map(histogram_general,input,h,w,inp)
      i=0
      for result in results:
        input[i]=result
        i=i+1
      
    return img

In [ ]:
import numpy as np




def cal_equalisation(img,ratio):
    Array = img * ratio
    Array = np.clip(Array, 0, 255)
    return Array




def RGB_equalisation(img):
    i =0
    img = np.float32(img)
    avg_RGB = []
    for i in range(3):
        avg = np.mean(img[:,:,i])
        avg_RGB.append(avg)
    # print('avg_RGB',avg_RGB)
    a_r = avg_RGB[0]/avg_RGB[2]
    a_g =  avg_RGB[0]/avg_RGB[1]
    ratio = [0,a_g,a_r]
    for i in range(1,3):
        img[:,:,i] = cal_equalisation(img[:,:,i],ratio[i])
    return img

In [ ]:
import os
import numpy as np
import cv2
import natsort
import xlwt
import datetime
import random

np.seterr(over='ignore')
if __name__ == '__main__':
    pass

# path = sett
pathinp = "/content/gdrive/My Drive/auv_files/train2/inp"
pathinp1 = "/content/gdrive/My Drive/auv_files/train2/oup"

filesinp = os.listdir(pathinp)
images=[]
# print(len(filesinp))
for i in range(len(filesinp)):
  imgdir = pathinp + '/' + filesinp[i]
  images.append(imgdir)
  # img = cv2.imread('InputImages/' + file)
  # import timeit
  # start = timeit.default_timer()
  # img = cv2.imread(imgdir)

  # sceneRadiance = RGB_equalisation(img)
  # sceneRadiance = stretching(sceneRadiance)
  # sceneRadiance = HSVStretching(sceneRadiance)
  # img_ = sceneRadianceRGB(sceneRadiance)

  # cv2.imwrite(pathinp1 + '/' + filesinp[i],img_)
  # cv2_imshow(img)
  # cv2_imshow(img_)
  # print(i)
  # stop = timeit.default_timer()
  # print('Time: ', stop - start)
print(len(images))  


701


In [ ]:
def process_image(imgdir):
  import timeit
  start = timeit.default_timer()
  img = cv2.imread(imgdir)

  sceneRadiance = RGB_equalisation(img)
  sceneRadiance = stretching(sceneRadiance)
  sceneRadiance = HSVStretching(sceneRadiance)
  img_ = sceneRadianceRGB(sceneRadiance)

  
  # cv2_imshow(img)
  # cv2_imshow(img_)
  
  stop = timeit.default_timer()
  print('Time: ', stop - start)
  return img_

In [ ]:
with concurrent.futures.ProcessPoolExecutor() as executor:
  results=executor.map(process_image,images)
  i=0
  for result in results:
    cv2.imwrite(pathinp1 + '/' + filesinp[i],result)
    print(i)
    i=i+1

Time:  5.097428316999867
Time:  5.198760423000749
0
1
Time:  5.124426494000545
Time:  5.115389977999257
2
3
Time:  5.058610764999685
Time:  5.125558628999897
4
5
Time:  5.112750975000381
Time:  5.11062267700072
6
7
Time:  5.407885349000026
Time:  5.311031611999169
8
9
Time:  5.132401009000205
Time:  5.252346162000322
10
11
Time:  5.080953412999406
Time:  5.042532520999885
12
13
Time:  5.070460556999933
Time:  5.129534486000011
14
15
Time:  5.083775486999912
16
Time:  5.110542102000181
17
Time:  5.1451418659999035
18
Time:  5.154840585999409
19
Time:  5.150246617999983
20
Time:  5.1683298150001065
21
Time:  5.183734373999869
Time:  5.13808923400029
22
23
Time:  5.115363317000629
24
Time:  5.144733303000066
25
Time:  5.1278396379993865
26
Time:  5.156750133999594
27
Time:  5.064997030999621
28
Time:  5.043270346000099
29
Time:  5.100657103999765
Time:  5.072584090999953
30
31
Time:  5.151391476999379
Time:  5.063843504000033
32
33
Time:  5.037495769000088
Time:  5.110847455000112
34
35
T